In [25]:
CURRENCY = "ethereum"
CURRENCY_SYMBOL = "ETH"
TWEETS_FOLDER    = f"data/crypto/{CURRENCY_SYMBOL}"
SEP_CHAR         = '~'
ENVS             = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] 
MAX_ROW_PER_FILE = 20000
query = f'#{CURRENCY} OR #{CURRENCY_SYMBOL}'

In [8]:
from twython import Twython

In [9]:
APP_KEY = 'mPQKoRwd2Pb9qpQyQmyG5s8KR'
APP_SECRET = 'HLvIhusvfzDLKaRXY8CnZGP143kp3E3f2KqQBIEMfVL5mOxZjq'
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1527859898}}

In [43]:
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm

In [42]:
NUMBER_OF_QUERIES = 450
data = {"statuses": []}
next_id = "998511795781361665"
with open(tweets_raw_file,"a+", encoding='utf-8') as f:
    if not next_id:
        f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n")
    while(True):
        twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
        last_size = 0
        for i in tqdm(range(NUMBER_OF_QUERIES)):
            if not next_id:
                data = twitter.search(q=query, lang='en', result_type='recent', count="100")
            else:
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100", max_id=next_id)["statuses"])
            if len(data["statuses"]) > 1:
                next_id = data["statuses"][len(data["statuses"]) - 1]['id']
            if last_size + 1 == len(data["statuses"]):
                break
            else:
                last_size = len(data["statuses"])

        print('Retrieved {0}, waiting for 15 minutes until next queries'.format(len(data["statuses"])))
        d = pd.DataFrame([[s["id"], s["text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["statuses"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        if last_size + 1 == len(data["statuses"]):
            print('No more new tweets, stopping...')
            break
        data["statuses"] = []
        
        sleep(910)

data/twitter/BTC/2018-05-29 12-20-53~2018-05-29 12-26-11.csv


In [5]:
import re 
from tqdm import tnrange, tqdm_notebook, tqdm

d = pd.read_csv(tweets_raw_file)
for i,s in enumerate(tqdm(d['Text'])):
    text = d.loc[i, 'Text']
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    d.loc[i, 'Text'] = text
f = open(tweets_clean_file, 'a+', encoding='utf-8')
d.to_csv(f, header=True, encoding='utf-8',index=False)

100%|██████████| 1120556/1120556 [17:30:30<00:00, 17.78it/s]  


In [6]:
df_clean = pd.read_csv(tweets_clean_file)
df_clean.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,1001439557504692224,RT : $btc bullish continuation,Tamekia Gay,0,548,0,Tue May 29 12:26:11 +0000 2018
1,1001439550860873728,RT : The Woman Powering Bitcoin - Interview Wi...,PiranhadoAmor,53,375,0,Tue May 29 12:26:09 +0000 2018
2,1001439550298828801,RT : Company Creates Decentralized VRWorld Whe...,🇻🇪 José Subero,2635,4,0,Tue May 29 12:26:09 +0000 2018
3,1001439549178961920,RT : Zynga poker chips are a fun currency in T...,Bingaman Sarah,1,890,0,Tue May 29 12:26:09 +0000 2018
4,1001439548877017088,RT Bitcoin: Bitcoin Cash is BitcoinVideo by Cr...,Crypto Br4in,9562,0,0,Tue May 29 12:26:08 +0000 2018


In [ ]:
df_clean.min(axis=0)

In [7]:
f.close()